<a href="https://colab.research.google.com/github/NehaChaudhary311/GPT2-Language-Model/blob/master/GPT2_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers

In [0]:
!git clone https://github.com/huggingface/transformers.git

In [0]:
cd transformers

In [0]:
pip install .

In [0]:
import torch
import torch.cuda

In [0]:
#Context-manager that changes the selected device.
dev = torch.device("cuda : 0" if torch.cuda.is_available() else "cpu")
#Returns the number of GPUs available.
n_gpu = torch.cuda.device_count()

In [0]:

import torch


In [0]:
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2') 
model.eval()

#Sentence taken from 
text = "Hello, have an embedding day."
#marked_text = "[CLS] " + text + " [SEP]"
# Tokenize our sentence with the GPT2 tokenizer.
tokenized_text = tokenizer.tokenize(text)
print("Tokenized text : " , tokenized_text)
tokenized_text_rem = tokenized_text.copy()

j = 0
#Removal of splitted tokens
for i in range(1, len(tokenized_text)):
  if tokenized_text[i] in ",:;(){}" :
            i += 1
  elif tokenized_text[i].startswith('Ġ'):
    i += 1 
  else :
    break
if i != len(tokenized_text):
  for j in range(i, len(tokenized_text)):
    if tokenized_text[j][0] in "Ġ.!?,":
      break
    else :
      j += 1
      
else : 
  pass
#tokenized_text_rem has the words after joining the splitted words      
tokenized_text_rem[i-1:j] = [''.join(tokenized_text_rem[i-1:j])]
print("Tokenized text after joining : " ,tokenized_text_rem)

#Encoding : maps words to key IDs in vocabulary ditionary
#Encoding our already tokenized text with the GPT2 encode()
text_index = tokenizer.encode(tokenized_text,add_prefix_space=True)
print("Encoded(tokenized text) : ", text_index)

#Finding
embeddings = model.transformer.wte.weight[text_index,:]
print("Word embedding : ", embeddings)
print(embeddings.shape)



#Taking average of to-be-removed-tokens only when 'i' is not equal to len(tokenized_text)
#That means 'i' has gone over the list and didn't find any splitted word, which further implies
#that we do no have to take average of the text_index

avg = torch.mean(embeddings[i-1:j], dim=0, keepdim=True)
embeddings = torch.cat([embeddings[:i-1], avg, embeddings[j:]], dim=0)
print(embeddings)
print(embeddings.shape)
  


Tokenized text :  ['Hello', ',', 'Ġhave', 'Ġan', 'Ġembed', 'ding', 'Ġday', '.']
Tokenized text after joining :  ['Hello', ',', 'Ġhave', 'Ġan', 'Ġembedding', 'Ġday', '.']
Encoded(tokenized text) :  [15496, 11, 423, 281, 11525, 12083, 1110, 13]
Word embedding :  tensor([[-0.0687, -0.1327,  0.0112,  ...,  0.0715, -0.0297, -0.0477],
        [ 0.0115, -0.0029,  0.0323,  ...,  0.0277, -0.0297, -0.0599],
        [ 0.0760,  0.0788,  0.1640,  ...,  0.0574, -0.0805,  0.0066],
        ...,
        [-0.0110, -0.1773,  0.1143,  ...,  0.1397,  0.3021,  0.1670],
        [-0.1379, -0.0294, -0.0026,  ..., -0.0966, -0.0726,  0.1160],
        [ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963]],
       grad_fn=<IndexBackward>)
torch.Size([8, 768])
tensor([[-0.0687, -0.1327,  0.0112,  ...,  0.0715, -0.0297, -0.0477],
        [ 0.0115, -0.0029,  0.0323,  ...,  0.0277, -0.0297, -0.0599],
        [ 0.0760,  0.0788,  0.1640,  ...,  0.0574, -0.0805,  0.0066],
        ...,
        [-0.0504, -0.0939,  0